In [150]:
%matplotlib inline
from importlib import reload
import utils; reload(utils)
from utils import *
from keras.layers.merge import add, multiply, dot, concatenate

In [71]:
path = "data/doubanmovie_dataset/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=64

# 读取数据

In [72]:
ratings = pd.read_csv(path + 'ratings.csv')
ratings.head()

,movieId,username,rating
0,26607296,ziegfelt,4
1,26607296,kaqee,4
2,26607296,mrhandsome,5
3,26607296,104816913,3
4,26607296,68707083,4


In [73]:
len(ratings)

245827

In [74]:
movies = pd.read_table(path + 'movie_info', sep = '|')
movies.head()

,id,name,zh_name,url,tag1,tag2,tag3,tag4,tag5,tag6,tag7,tag8
0,25730878,Il giovane favoloso,青春韶华 / Leopardi,http://movie.douban.com/movie/25730878,意大利,传记,意大利电影,2014,电影,MarioMartone,历史,Mario_Martone
1,25845910,小门神,Little Door Gods,http://movie.douban.com/movie/25845910,动画,国产动画,中国,小门神,奇幻,2016,追光动画,大陆
2,22519250,The One I Love,爱的就是你 / 我爱的那个人,http://movie.douban.com/movie/22519250,爱情,美国,悬疑,2014,科幻,剧情,惊悚,奇幻
3,25844867,Night Shift,NaN,http://movie.douban.com/movie/25844867,HKTV,港剧,悬疑,电视剧,2015,TVB,黑夜,香港
4,26355630,Quantico,谍网 / 新生无间道,http://movie.douban.com/movie/26355630,美剧,间谍,ABC,2015,犯罪,美国,推理,电视剧


In [75]:
id2names = movies.set_index('id')['zh_name'].to_dict

In [76]:
movies = ratings.movieId.unique()
users = ratings.username.unique()
len(movies), len(users)

(1959, 87782)

In [77]:
ratings.movieId.head(), ratings.username.head()

(0    26607296
 1    26607296
 2    26607296
 3    26607296
 4    26607296
 Name: movieId, dtype: int64, 0      ziegfelt
 1         kaqee
 2    mrhandsome
 3     104816913
 4      68707083
 Name: username, dtype: object)

In [78]:
movieid2idx = {o:i for i, o in enumerate(movies)}
username2idx = {o:i for i, o in enumerate(users)}

In [79]:
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
# ratings["userId"] = pd.Series(ratings.username.apply(lambda x: username2idx[x]))

ratings = ratings.assign(userId = pd.Series(ratings.username.apply(lambda x: username2idx[x])).values)

In [80]:
ratings.head()

,movieId,username,rating,userId
0,0,ziegfelt,4,0
1,0,kaqee,4,1
2,0,mrhandsome,5,2
3,0,104816913,3,3
4,0,68707083,4,4


In [81]:
# n_users, n_movies = len(users), len(movies)
n_users, n_movies = ratings.userId.nunique(), ratings.movieId.nunique()
n_users, n_movies

(87782, 1959)

In [82]:
n_factor = 50

In [83]:
np.random.seed = 42

把数据分成train和valid

In [84]:
mask = np.random.rand(len(ratings)) < 0.01
train = ratings[mask]
valid = ratings[~mask]

In [85]:
len(train)

2514

In [96]:
n_users, n_movies

(87782, 1959)

In [134]:
user_in = Input(shape = (1,), dtype = 'int64', name = 'user_in')
u = Embedding(n_users, n_factor, input_length=1, embeddings_regularizer=l2(1e-4))(user_in)
movie_in = Input(shape = (1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factor, input_length=1, embeddings_regularizer=l2(1e-4))(movie_in)

In [135]:
# type(keras.layers.merge.Add)
type(keras.layers.embeddings)
# type(keras.layers.merge) #奇怪，embeddings, recurrent等都是module，merge居然是function？！！！！

module

In [137]:
u.shape

TensorShape([Dimension(None), Dimension(1), Dimension(50)])

In [138]:
x = dot([u, m], 2)
x.shape

TensorShape([Dimension(None), Dimension(1), Dimension(1)])

In [139]:
x = dot([u, m], 2)
x = Flatten()(x)
model = Model(inputs = [user_in, movie_in], outputs = x)
model.compile(Adam(0.01), loss = 'mse')

In [140]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_in (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
movie_in (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_17 (Embedding)         (None, 1, 50)         4389100                                      
____________________________________________________________________________________________________
embedding_18 (Embedding)         (None, 1, 50)         97950                                        
___________________________________________________________________________________________

In [141]:
model.fit([train.userId, train.movieId], train.rating, epochs=1)

Epoch 1/1
2514/2514 [==============================] - 7s - loss: 12.2268      

In [117]:
model.predict([valid.userId, valid.movieId])

array([[ -3.1357e-05],
       [  2.8856e-05],
       [ -1.8218e-05],
       ..., 
       [ -2.1895e-06],
       [ -1.1007e-06],
       [  5.9854e-07]], dtype=float32)

In [119]:
user_in = Input(shape = (1,), dtype = 'int64', name = 'user_in')
u = Embedding(n_users, n_factor, input_length=1, embeddings_regularizer=l2(1e-4))(user_in)
movie_in = Input(shape = (1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factor, input_length=1, embeddings_regularizer=l2(1e-4))(movie_in)

ub_in = Embedding(n_users, 1, input_length=1)(user_in)
ub = Flatten()(ub_in)
mb_in = Embedding(n_movies, 1, input_length=1)(movie_in)
mb = Flatten()(mb_in)

In [131]:
x = dot([u, m], 2)
x = Flatten()(x)
x = add([x, ub])
x = add([x, mb])
model = Model(inputs = [user_in, movie_in], outputs = x)
model.compile(Adam(0.01), loss = 'mse')

In [132]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_in (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
movie_in (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_13 (Embedding)         (None, 1, 50)         4389100                                      
____________________________________________________________________________________________________
embedding_14 (Embedding)         (None, 1, 50)         97950                                        
___________________________________________________________________________________________

In [147]:
model.fit([train.userId, train.movieId], train.rating, epochs=1, validation_data=([valid.userId, valid.movieId], valid.rating))

Train on 2514 samples, validate on 243313 samples
Epoch 1/1
2514/2514 [==============================] - 57s - loss: 11.6533 - val_loss: 12.4050

# 用神经网络

In [149]:
user_in = Input(shape = (1,), dtype = 'int64', name = 'user_in')
u = Embedding(n_users, n_factor, input_length=1, embeddings_regularizer=l2(1e-4))(user_in)
movie_in = Input(shape = (1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factor, input_length=1, embeddings_regularizer=l2(1e-4))(movie_in)

In [153]:
x = concatenate([u, m], 2)
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn = Model([user_in, movie_in], x)
nn.compile(Adam(0.001), loss='mse')

In [154]:
model.fit([train.userId, train.movieId], train.rating, epochs = 1, validation_split=0.2)

Train on 2011 samples, validate on 503 samples
Epoch 1/1
2011/2011 [==============================] - 6s - loss: 8.2396 - val_loss: 6.9716